# I. Make Document-Term Matrix (Basic)
* 장점 : 학습용(과정 이해)
* 단점 : 코드가 복잡

In [1]:
from konlpy.tag import *
import numpy as np

In [2]:
corpus = [
    "고인물은 게임을 즐긴다.",
    "남중호는 게임을 잘한다.",
    "남중구는 게임을 못한다.",
    "나는 나를 사랑한다",
    "너의 게임 나의 게임 사랑하라"
]
s=" ".join(corpus)

print('All string :' ,s)

All string : 고인물은 게임을 즐긴다. 남중호는 게임을 잘한다. 남중구는 게임을 못한다. 나는 나를 사랑한다 너의 게임 나의 게임 사랑하라


In [3]:
# posToUse=["NNP","NNG","MAG","NP","VV","VV+EF"]
posToUse=["NNP","NNG","NP"]
def getTokens(s):
    global posToUse
    return " ".join([ i[0] for i in  Mecab().pos(s) if i[1] in posToUse ] )
    
print('All tockents:', getTokens(s))  

All tockents: 인물 게임 남 중호 게임 남중구 게임 나 나 사랑 너 게임 나 게임 사랑


In [4]:
# Document Frequency
from collections import Counter
counts=Counter(getTokens(s).split())
counts_sorted=sorted(counts, key=counts.get,reverse=True)
for i in counts_sorted:
    print("%10s %2d"%(i,counts[i]))


        게임  5
         나  3
        사랑  2
        인물  1
         남  1
        중호  1
       남중구  1
         너  1


## word2idx

In [5]:
word2idx = {word: ii for ii, word in enumerate(counts_sorted,1)}
word2idx

{'게임': 1, '나': 2, '사랑': 3, '인물': 4, '남': 5, '중호': 6, '남중구': 7, '너': 8}

## ind2word

In [6]:
ind2word = {ii: word for ii, word in enumerate(counts_sorted,1)}
ind2word

{1: '게임', 2: '나', 3: '사랑', 4: '인물', 5: '남', 6: '중호', 7: '남중구', 8: '너'}

## Document Term Frequency Matrix

In [7]:
def getDtm(corpus,word2idx):
    dicCount=len(word2idx)
    cv_org=np.zeros(dicCount, dtype=np.int)
    dtm=[(i, getTokens(doc),cv_org.copy()) for i,doc in enumerate(corpus)]
    for r in dtm:
        cv=r[2]
        counts=Counter(r[1].split())
        for k in counts.keys():
            dtm[r[0]][2][word2idx[k]-1]=dtm[r[0]][2][word2idx[k]-1]+counts[k]
#         print(r[2])    
    return dtm

getDtm(corpus,word2idx)
# print('DTM: ',vect.vocabulary_)

[(0, '인물 게임', array([1, 0, 0, 1, 0, 0, 0, 0])),
 (1, '남 중호 게임', array([1, 0, 0, 0, 1, 1, 0, 0])),
 (2, '남중구 게임', array([1, 0, 0, 0, 0, 0, 1, 0])),
 (3, '나 나 사랑', array([0, 2, 1, 0, 0, 0, 0, 0])),
 (4, '너 게임 나 게임 사랑', array([2, 1, 1, 0, 0, 0, 0, 1]))]

# II. Document-Term Matrix by scikit-learn
* 장점 : 구현이 쉽다.
* 주의 : StopWord나 글자 1글자 단어가 되려면 tokenizer 옵션 필요

## 0. Very Simple Version

In [8]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

# cf) for English : word2idx  
vect = CountVectorizer()
vect.fit(['a aa aaa b bb bbb 나 나나 나나나'])
getTokens(s)
vect.vocabulary_

{'aa': 0, 'aaa': 1, 'bb': 2, 'bbb': 3, '나나': 4, '나나나': 5}

## 준비

In [9]:
corpus = [
    "고인물은 게임을 즐긴다.",
    "남중호는 게임을 잘한다.",
    "남중구는 게임을 못한다.",
    "나는 나란 놈 사랑한다.18",
    "너의 게임 나의 게임 사랑하라"
]
s=" ".join(corpus)

print('All string :' ,s)

# posToUse=["NNP","NNG","MAG","NP","VV","VV+EF","IC"]
posToUse=["NNP","NNG","NP","NNB"]

def getTokens(s):
    global posToUse
    return [ i[0] for i in  Mecab().pos(s) if i[1] in posToUse ] 
    
print('All tockents:', getTokens(s))

All string : 고인물은 게임을 즐긴다. 남중호는 게임을 잘한다. 남중구는 게임을 못한다. 나는 나란 놈 사랑한다.18 너의 게임 나의 게임 사랑하라
All tockents: ['인물', '게임', '남', '중호', '게임', '남중구', '게임', '나', '나', '놈', '사랑', '너', '게임', '나', '게임', '사랑']


## 1. Simple Version

In [10]:
vect = CountVectorizer()
vect.fit(getTokens(s))
print('Dictionary : ',vect.vocabulary_)

vect.transform([" ".join(getTokens(i)) for i in corpus]).toarray()

Dictionary :  {'인물': 3, '게임': 0, '중호': 4, '남중구': 1, '사랑': 2}


array([[1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [2, 0, 1, 0, 0]])

## 2. tokenizer 사용 Version

In [11]:
posToUse=["NNP","NNG","NP","NNB"]
vect = CountVectorizer(tokenizer=getTokens)
vect.fit(getTokens(s))
print('Dictionary : ',vect.vocabulary_)
vect.transform([" ".join(getTokens(i)) for i in corpus]).toarray()

Dictionary :  {'인물': 7, '게임': 0, '남': 2, '중호': 8, '남중구': 3, '나': 1, '놈': 5, '사랑': 6, '너': 4}


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


array([[1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 1, 0, 0],
       [2, 1, 0, 0, 1, 0, 1, 0, 0]])

## 3. tokenizer+stopword 적용 Version


In [12]:
posToUse=["NNP","NNG","NP","NNB"]
stopwords=["놈","18"]
def getToken(s,pos=posToUse,stopword=stopwords):
    return [ w for w,t in  Mecab().pos(s) if t in pos and w not in stopword ] 
vect = CountVectorizer(tokenizer=getToken)
vect.fit(getTokens(s))
print('Dictionary : ',vect.vocabulary_)
vect.transform([" ".join(getTokens(i)) for i in corpus]).toarray()

Dictionary :  {'인물': 6, '게임': 0, '남': 2, '중호': 7, '남중구': 3, '나': 1, '사랑': 5, '너': 4}


array([[1, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0],
       [2, 1, 0, 0, 1, 1, 0, 0]])